# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Thoại Đăng Khoa
- MSSV: 20127043
- Lớp: 20CLC08

# Import

In [1]:
import pandas as pd
import numpy as np
import copy

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# convert to numpy array
X_train = X_train.to_numpy() 
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# Sinh viên có thể sử dụng các khác nếu cần

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

# Cài đặt hàm

In [ ]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

# Cài đặt hàm RMSE
def rmse(y, y_hat):
    return np.sqrt(np.mean((y.ravel() - y_hat.ravel())**2))

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [ ]:
# Phần code cho yêu cầu 1a
# init lr
lr = OLSLinearRegression().fit(X_train, y_train)

# print feature vector
print("Feature vectors W: ", lr.get_params())

# predict
y_test_predict = lr.predict(X_test)

# print result
print('-------------------------------------------')
prediction = pd.DataFrame({'Test': y_test, 'Predicted': y_test_predict})
print(prediction)

In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
print("RMSE on all features model: ", rmse(y_test, y_test_predict))

Công thức hồi quy

$$\text{Life expectancy} = 0.015101 * Adult Mortality + 0.090220 * BMI + 0.042922 * Polio + 0.139289 * Diphtheria - 0.567333 * HIV/AIDS - 0.000101 * GDP + 0.740713 * ThinnessAge(10-19) + 0.190936 * ThinessAge(5-9) + 24.505974 * IncomeCompositionOf Resources + 2.393517 * Schooling$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [ ]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

# make deep copy
X_train_clone = copy.deepcopy(X_train)
y_train_clone = copy.deepcopy(y_train)

# shuffle dataset
shuffler = np.random.permutation(len(X_train_clone))
X_train_clone = X_train_clone[shuffler]
y_train_clone  = y_train_clone[shuffler]

cross = []
list_of_column_names = list(train.columns) # array contains features name
RMSE = np.zeros(10) #initialize RMSE array

for i in range(0, 10):
    X_train_feature = X_train_clone[:,i]
    y_train_feature = y_train_clone.copy()
    
    for k in range(0, 5):
        lr = OLSLinearRegression()
        
        # create test dataset
        X_k_val = X_train_feature[217*k:217*k + 217]
        y_k_val = y_train_feature[217*k:217*k + 217]
        
        # create training dataset
        if(k == 0):  
            X_k_train = X_train_feature[217:] # find x_k_train
            y_k_train = y_train_feature[217:] # find y_k_train
            
        else: 
            # find x_k_train
            X_k_train_1 = X_train_feature[0:217*k]
            X_k_train_2 = X_train_feature[217*k + 217:]
            X_k_train = np.concatenate((X_k_train_1,X_k_train_2), axis = 0)
            
            # find y_k_train
            y_k_train_1 = y_train_feature[0:217*k]
            y_k_train_2 = y_train_feature[217*k + 217:]
            y_k_train = np.concatenate((y_k_train_1,y_k_train_2), axis = 0)
            
       # convert to 2d array
        X_k_train = X_k_train.reshape(-1,1) 
        y_k_train = y_k_train.reshape(-1,1)
        X_k_val = X_k_val.reshape(-1,1)
        y_k_val = y_k_val.reshape(-1,1)
        
        # training
        lr.fit(X_k_train, y_k_train)
        # predict
        y_k_val_pred = lr.predict(X_k_val)
        RMSE[i] += rmse(y_k_val, y_k_val_pred)
        
    # calculate mean rmse i_th
    RMSE[i] /= 5
    cross.append([list_of_column_names[i],RMSE[i]])
    

# print cross validation    
print("Cross-validation: ", RMSE)
print("-----------------------------------------------------------------------------------------------")

# print report table
prediction = pd.DataFrame(cross,columns=['Model with one feature','RMSE'])
print(prediction)
print("-----------------------------------------------------------------------------------------------")

# find the best feature
RMSE = list(RMSE)
min_rmse = min(RMSE) # get the minimum rmse value in the list

# return the index of minimum rmse value 
min_index=RMSE.index(min_rmse)

# print the best feature
print("The best feature: ", list_of_column_names[min_index])

In [ ]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_train_best_feature = X_train[:,min_index].reshape(-1,1)
lr = OLSLinearRegression().fit(X_train_best_feature, y_train)

# print feature vector
print("Feature vector W: ", lr.get_params())

In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
X_test_best_feature = X_test[:,min_index].reshape(-1,1)
y_test_predict_best_feature = lr.predict(X_test_best_feature)
print("RMSE on best feature model: ", rmse(y_test, y_test_predict_best_feature))

Công thức hồi quy

$$\text{Life expectancy} = 5.5573994*Schooling$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

# Normalize the dataset
- Normalize the dataset to find feature importance
- Find importance value for each feature based on coefficient and standard deviation
- Scale that importance values in range of [0,100]

In [ ]:
# Find 3 features importance
stdev = []
x = train[["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9",
       "Income composition of resources", "Schooling"]]
y = train["Life expectancy"]

lr = OLSLinearRegression().fit(X_train, y_train)

lr_coef = lr.get_params() # get feature vector

tables = pd.DataFrame(lr_coef, x.columns, columns=['coefficient'])
tables.coefficient = tables.coefficient.abs()

# Transforming the units of coefficients

# calculate standard deviation for each feature
stdeviations = []
for i in x.columns:
    stdev = train[i].std()
    stdeviations.append(stdev)

tables["standard_deviation"] = np.array(stdeviations).reshape(-1,1)
tables["importance"] = tables["coefficient"] * tables["standard_deviation"]

# normalize the importance column in range of [0, 100].
tables['importance_normalized (0 - 100)'] = 100*tables['importance'] / tables['importance'].max()

# display tables
tables.head(10) 

# Find 4 features importance
- Find indexes of 4 features importance
- Use that indexes to retrieve columns of corresponding features
- Building models based on the corresponding features

In [ ]:
# convert importance_normalized to numpy array
importance_rates = tables['importance_normalized (0 - 100)'].to_numpy()

# find index of 4 features importance
biggest_indexes = np.argpartition(importance_rates, -4)[-4:] # find 4 biggest value's index

# find 4 most important features columns based on index
indexes_labels = x.columns[biggest_indexes]

# display table
best_features_table = pd.DataFrame(importance_rates[biggest_indexes],indexes_labels, columns=['Importance (0-100)'])
best_features_table["Index column"] = biggest_indexes
best_features_table.head()

# Build model 1
- 2 most important features: Schooling (9), Income composition of resources (8)

In [ ]:
def Model_1(X_train_clone,y_train_clone):
    
    RMSE = 0 #initialize RMSE value

    X_train_feature = np.concatenate([X_train_clone[:,9].reshape(-1,1),X_train_clone[:,8].reshape(-1,1)], axis = 1)
    y_train_feature = y_train_clone.copy()

    for k in range(0, 5):
        lr = OLSLinearRegression()

        # create test dataset
        X_k_val = X_train_feature[217*k:217*k + 217,:]
        y_k_val = y_train_feature[217*k:217*k + 217]

        # create training dataset
        if(k == 0):  
            X_k_train = X_train_feature[217:,:] # find x_k_train
            y_k_train = y_train_feature[217:] # find y_k_train

        else: 
            # find x_k_train
            X_k_train_1 = X_train_feature[0:217*k,:]
            X_k_train_2 = X_train_feature[217*k + 217:,:]
            X_k_train = np.concatenate((X_k_train_1,X_k_train_2), axis = 0)

            # find y_k_train
            y_k_train_1 = y_train_feature[0:217*k]
            y_k_train_2 = y_train_feature[217*k + 217:]
            y_k_train = np.concatenate((y_k_train_1,y_k_train_2), axis = 0)

        # training
        lr.fit(X_k_train, y_k_train)
        # predict
        y_k_val_pred = lr.predict(X_k_val)
        # calculate RMSE k_th
        RMSE += rmse(y_k_val, y_k_val_pred)
    # calculate mean RMSE
    return RMSE / 5


# Build model 2
- 3 most important features: Schooling (9), Income composition of resources (8), Thinness age 10-19 (6)

In [ ]:
def Model_2(X_train_clone, y_train_clone):
    RMSE = 0 #initialize RMSE value

    X_train_feature = np.concatenate([X_train_clone[:,9].reshape(-1,1),
            X_train_clone[:,8].reshape(-1,1),X_train_clone[:,6].reshape(-1,1)], axis = 1)
    y_train_feature = y_train_clone.copy()

    for k in range(0, 5):
        lr = OLSLinearRegression()

        # create test dataset
        X_k_val = X_train_feature[217*k:217*k + 217,:]
        y_k_val = y_train_feature[217*k:217*k + 217]

        # create training dataset
        if(k == 0):  
            X_k_train = X_train_feature[217:,:] # find x_k_train
            y_k_train = y_train_feature[217:] # find y_k_train

        else: 
            # find x_k_train
            X_k_train_1 = X_train_feature[0:217*k,:]
            X_k_train_2 = X_train_feature[217*k + 217:,:]
            X_k_train = np.concatenate((X_k_train_1,X_k_train_2), axis = 0)

            # find y_k_train
            y_k_train_1 = y_train_feature[0:217*k]
            y_k_train_2 = y_train_feature[217*k + 217:]
            y_k_train = np.concatenate((y_k_train_1,y_k_train_2), axis = 0)

        # training
        lr.fit(X_k_train, y_k_train)
        # predict
        y_k_val_pred = lr.predict(X_k_val)
        # calculate RMSE k_th
        RMSE += rmse(y_k_val, y_k_val_pred)
    # calculate mean RMSE 
    return RMSE / 5


# Build model 3
- 4 most important features: Schooling (9), Income composition of resources (8), Thinness age 10-19 (6), Diphtheria (3)

In [ ]:
def Model_3(X_train_clone, y_train_clone):

    RMSE = 0 #initialize RMSE value

    X_train_feature = np.concatenate([X_train_clone[:,9].reshape(-1,1),
            X_train_clone[:,8].reshape(-1,1),X_train_clone[:,6].reshape(-1,1),X_train_clone[:,3].reshape(-1,1)], axis = 1)
    y_train_feature = y_train_clone.copy()

    for k in range(0, 5):
        lr = OLSLinearRegression()

        # create test dataset
        X_k_val = X_train_feature[217*k:217*k + 217,:]
        y_k_val = y_train_feature[217*k:217*k + 217]

        # create training dataset
        if(k == 0):  
            X_k_train = X_train_feature[217:,:] # find x_k_train
            y_k_train = y_train_feature[217:] # find y_k_train

        else: 
            # find x_k_train
            X_k_train_1 = X_train_feature[0:217*k,:]
            X_k_train_2 = X_train_feature[217*k + 217:,:]
            X_k_train = np.concatenate((X_k_train_1,X_k_train_2), axis = 0)

            # find y_k_train
            y_k_train_1 = y_train_feature[0:217*k]
            y_k_train_2 = y_train_feature[217*k + 217:]
            y_k_train = np.concatenate((y_k_train_1,y_k_train_2), axis = 0)

        # training
        lr.fit(X_k_train, y_k_train)
        # predict
        y_k_val_pred = lr.predict(X_k_val)
        # calculate RMSE k_th
        RMSE += rmse(y_k_val, y_k_val_pred)
    # calculate mean RMSE
    return RMSE / 5


# Build model 4
- 4 features (with new feature added): Schooling (9), Income composition of resources (8), Thinness age 10-19 (6), Vaccination rate among 1-year-olds (10)

# Create new feature Vaccination rate among 1-year-olds


In [ ]:
def reportNewFeature(X_train_clone, y_train_clone):
    # create new feature Vaccination_rate_1_year_olds based on Polio and Diphtheria
    new_feature = (X_train_clone[:,2] + X_train_clone[:,3]) / 2

    X_train_clone = np.append(X_train_clone, new_feature.reshape(-1,1), axis = 1)

    lr = OLSLinearRegression().fit(X_train_clone, y_train_clone)

    # create report table
    dict = {'Features' : ["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9",
           "Income composition of resources", "Schooling", "Vaccination rate 1-year-olds"],
            'coeff' : lr.get_params()
           }

    df = pd.DataFrame(dict)
    df['coeff'] = df['coeff'].abs()

    # calcualte standard deviation for each feature
    stdeviations = []
    for i in range(0,11):
        stdev = np.std(X_train_clone[:,i])
        stdeviations.append(stdev)

    df["std"] = np.array(stdeviations).reshape(-1,1)
    # normalize the importance column in range of [0, 100].
    df['importance'] = 100*(df["coeff"] * df["std"]) / (df["coeff"] * df["std"]).max()
    print(df)


# make deep copy
X_train_clone = copy.deepcopy(X_train)
y_train_clone = copy.deepcopy(y_train)

# print report
print("Report new feature: ")
reportNewFeature(X_train_clone, y_train_clone)
    

In [ ]:
def Model_4(X_train_clone, y_train_clone):
    # create new feature based on Polio And dipl
    new_feature = (X_train_clone[:,2] + X_train_clone[:,3]) / 2

    X_train_clone = np.append(X_train_clone, new_feature.reshape(-1,1), axis = 1)

    lr = OLSLinearRegression().fit(X_train_clone, y_train_clone)


    RMSE = 0 #initialize RMSE value

    # create X_train_feature, y_train_feature
    X_train_feature = np.concatenate([X_train_clone[:,9].reshape(-1,1),
            X_train_clone[:,8].reshape(-1,1),X_train_clone[:,6].reshape(-1,1),
            X_train_clone[:,10].reshape(-1,1)], axis = 1)
   
    y_train_feature = y_train_clone.copy()

    for k in range(0, 5):
        lr = OLSLinearRegression()

        # create test dataset
        X_k_val = X_train_feature[217*k:217*k + 217,:]
        y_k_val = y_train_feature[217*k:217*k + 217]

        # create training dataset
        if(k == 0):  
            X_k_train = X_train_feature[217:,:] # find x_k_train
            y_k_train = y_train_feature[217:] # find y_k_train

        else: 
            # find x_k_train
            X_k_train_1 = X_train_feature[0:217*k,:]
            X_k_train_2 = X_train_feature[217*k + 217:,:]
            X_k_train = np.concatenate((X_k_train_1,X_k_train_2), axis = 0)

            # find y_k_train
            y_k_train_1 = y_train_feature[0:217*k]
            y_k_train_2 = y_train_feature[217*k + 217:]
            y_k_train = np.concatenate((y_k_train_1,y_k_train_2), axis = 0)

        # training
        lr.fit(X_k_train, y_k_train)
        # predict
        y_k_val_pred = lr.predict(X_k_val)
        # calculate RMSE k_th
        RMSE += rmse(y_k_val, y_k_val_pred)
    # calculate mean rmse i_th
    return RMSE / 5


In [ ]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

# make deep copy
X_train_clone = copy.deepcopy(X_train)
y_train_clone = copy.deepcopy(y_train)

# shuffle dataset once
shuffler = np.random.permutation(len(X_train_clone))
X_train_clone = X_train_clone[shuffler]
y_train_clone  = y_train_clone[shuffler]

# test on all models
firstModel = Model_1(X_train_clone, y_train_clone)
secondModel = Model_2(X_train_clone, y_train_clone)
thirdModel = Model_3(X_train_clone, y_train_clone)
fourthModel = Model_4(X_train_clone, y_train_clone)

# print result
print("RMSE of first model: ", firstModel)
print("RMSE of second model: ", secondModel)
print("RMSE of third model: ", thirdModel)
print("RMSE of fourth model: ", fourthModel)
print("Cross-validation: ", [firstModel,secondModel,thirdModel,fourthModel])

In [ ]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_best_model = np.concatenate([X_train[:,9].reshape(-1,1),
            X_train[:,8].reshape(-1,1),X_train[:,6].reshape(-1,1),X_train[:,3].reshape(-1,1)], axis = 1)
lr = OLSLinearRegression().fit(X_best_model, y_train)

# print feature vector
print("Feature vectors W: ", lr.get_params())

In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
X_test_best_model = np.concatenate([X_test[:,9].reshape(-1,1),
            X_test[:,8].reshape(-1,1),X_test[:,6].reshape(-1,1),X_test[:,3].reshape(-1,1)], axis = 1)

y_test_predict_best_model = lr.predict(X_test_best_model)

# print result
print("RMSE on my best model: ", rmse(y_test, y_test_predict_best_model))

Công thức hồi quy

$$\text{Life expectancy} = 2.39 * Schooling + 29.55 * IncomeCompositionOfResource + 0.93 * ThinnessAge(10-19) + 0.19 * Diphtheria$$